In [ ]:
import sys
import os
import pandas as pd

notebook_dir = os.getcwd()
sys.path.append(os.path.join(notebook_dir, "..", "code"))
import utils

# Import DataFrames

In [4]:
df_im = pd.read_parquet("../sourcedata/clips_metadata_from_im.parquet")
df_base = pd.read_parquet("../sourcedata/clips_metadata_with_patterns.parquet")
df_im = df_im.rename(columns={"StateClipCode": "ClipCode"})

df_base['Cleared'] = df_base['Cleared'].map({'True': 1, 'False': 0})
df = pd.concat([df_base, df_im], ignore_index=True)
df['Scene'] = df['Scene'].astype(int)
df['Average_speed'] = df['Average_speed'].astype(float)
df['Duration'] = df['Duration'].astype(float)
df['X_Traveled'] = df['X_Traveled'].astype(float)

df_variables_hum = pd.read_parquet("../sourcedata/df_variables_hum.parquet")
df_variable_ppo= pd.read_parquet("../sourcedata/df_variables_ppo.parquet")

df_variables = pd.concat([df_variables_hum, df_variable_ppo], ignore_index=True)

df_variables['player_x_pos'] = df_variables['player_x_posHi']*255 + df_variables['player_x_posLo']
df_variables['Scene'] = df_variables['Scene'].astype(int)


In [5]:
df_meta = df[df['Subject'].str.startswith('sub-')].groupby(['SceneFullName'])

full_scenes = []
scenes_to_drop = []
for scene, df_scene in df_meta:
    if df_scene['Subject'].nunique() < 5:
        scenes_to_drop.append(scene if isinstance(scene, str) else scene[0])
    else:
        full_scenes.append(scene if isinstance(scene, str) else scene[0])

print(f"Number of full scenes: {len(full_scenes)}")

Number of full scenes: 290


In [6]:
df = df[~df['SceneFullName'].isin(scenes_to_drop)]
df_variables = df_variables[~df_variables['SceneFullName'].isin(scenes_to_drop)]

mask_im = df['Model'].str.startswith('sub-0')
df.loc[mask_im, 'Learning_Phase'] = df.loc[mask_im, 'Model'].str[:]
df.loc[mask_im, 'Subject'] = "im_"+df.loc[mask_im, 'Model'].str[:6]
df.loc[mask_im, 'SceneFullName'] = df.loc[mask_im, 'World'].astype(str)+'-'+df.loc[mask_im, 'Level'].astype(str)+'-'+df.loc[mask_im, 'Scene'].astype(str)

mask_ppo = df['Model'].str.startswith('ep')
df.loc[mask_ppo, 'Learning_Phase'] = df.loc[mask_ppo, 'Model'].str[:]
df.loc[mask_ppo, 'Subject'] = "ppo"
df.loc[mask_ppo, 'Average_speed'] = df.loc[mask_ppo, 'X_Traveled'] / df.loc[mask_ppo, 'Duration']

range = 58

## Top Clearance Deltas for Humans (delta total)

In [43]:
df_meta = df[df['Model']=='human'].groupby(['SceneFullName'])
columns = ['sub-01','sub-02', 'sub-03', 'sub-05', 'sub-06', 'delta_mean' ]
indexs = list(df_meta.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

order_4phases = ["Early discovery", "Late discovery", "Early practice", "Late practice"]
phase_to_human = {p: i % 4 for i, p in enumerate(order_4phases)}

for scene, df_scene in df_meta:

    df_clearance = df_scene.groupby(["Subject", "Learning_Phase"])["Cleared"].mean().reset_index(name="Cleared")
    df_delta_tot = utils.compute_delta_tot(df_clearance, phase_to_human, 'Cleared')

    delta_mean = {'delta_mean': df_delta_tot['delta_tot'].mean().item()}
    delta_sub = dict(zip(df_delta_tot['Subject'], df_delta_tot['delta_tot']))
    row = {**delta_sub, **delta_mean}

    scene_name = scene if isinstance(scene, str) else scene[0]

    df_deltas_tot.loc[scene_name] = row

df_deltas_tot = df_deltas_tot.sort_values('delta_mean', ascending=False)
hum_top_clr = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(range)

Returns hum_top_clr: a list of the top $range scenes in terms of delta clearance for the humans.

## Top Speed Deltas for Humans (delta total)

In [44]:
df_meta = df[df['Model']=='human'].groupby(['SceneFullName'])
columns = ['sub-01','sub-02', 'sub-03', 'sub-05', 'sub-06', 'delta_mean' ]
indexs = list(df_meta.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

order_4phases = ["Early discovery", "Late discovery", "Early practice", "Late practice"]
phase_to_human = {p: i % 4 for i, p in enumerate(order_4phases)}

for scene, df_scene in df_meta:

    df_clearance = df_scene.groupby(["Subject", "Learning_Phase"])["Average_speed"].mean().reset_index(name="Average_speed")
    df_delta_tot = utils.compute_delta_tot(df_clearance, phase_to_human, 'Average_speed')

    delta_mean = {'delta_mean': df_delta_tot['delta_tot'].mean().item()}
    delta_sub = dict(zip(df_delta_tot['Subject'], df_delta_tot['delta_tot']))
    row = {**delta_sub, **delta_mean}

    scene_name = scene if isinstance(scene, str) else scene[0]

    df_deltas_tot.loc[scene_name] = row

df_deltas_tot = df_deltas_tot.sort_values('delta_mean', ascending=False)
hum_top_spd = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(range)

Returns hum_top_spd: a list of the top $range scenes in terms of delta average speed for the humans.

## Top Mean MAD for Humans

In [ ]:
df_var = df_variables[df_variables['Subject'].str.startswith('sub-')].groupby(['SceneFullName'])
columns = ['sub-01','sub-02', 'sub-03', 'sub-05', 'sub-06', 'MAD_mean' ]
indexs = list(df_var.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

for scene, df_scene in df_var:

    df_delta_tot = utils.get_mads(df_scene).groupby('Subject')["MAD_mean"].mean().reset_index(name='MAD_mean')
    delta_mean = {'MAD_mean': df_delta_tot['MAD_mean'].mean().item()}
    delta_sub = dict(zip(df_delta_tot['Subject'], df_delta_tot['MAD_mean']))
    row = {**delta_sub, **delta_mean}

    scene_name = scene if isinstance(scene, str) else scene[0]

    df_deltas_tot.loc[scene_name] = row

df_deltas_tot = df_deltas_tot.sort_values('MAD_mean', ascending=False)
hum_top_mad = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(5)

Returns hum_top_mad: a list of the top $range scenes in terms of mean MAD for the humans.

## Bottom Clearance Deltas for Humans (delta total)

In [70]:
df_meta = df[df['Model']=='human'].groupby(['SceneFullName'])
columns = ['sub-01','sub-02', 'sub-03', 'sub-05', 'sub-06', 'delta_mean' ]
indexs = list(df_meta.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

order_4phases = ["Early discovery", "Late discovery", "Early practice", "Late practice"]
phase_to_human = {p: i % 4 for i, p in enumerate(order_4phases)}

for scene, df_scene in df_meta:

    df_clearance = df_scene.groupby(["Subject", "Learning_Phase"])["Cleared"].mean().reset_index(name="Cleared")
    df_delta_tot = utils.compute_delta_tot(df_clearance, phase_to_human, 'Cleared')

    delta_mean = {'delta_mean': df_delta_tot['delta_tot'].mean().item()}
    delta_sub = dict(zip(df_delta_tot['Subject'], df_delta_tot['delta_tot']))
    row = {**delta_sub, **delta_mean}

    scene_name = scene if isinstance(scene, str) else scene[0]

    df_deltas_tot.loc[scene_name] = row

df_deltas_tot = df_deltas_tot.sort_values('delta_mean', ascending=True)
hum_bot_clr = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(5)

Returns hum_bot_clr: a list of the bottom $range scenes in terms of delta clearance for the humans.

## Bottom Speed Deltas for Humans (delta total)

In [ ]:
df_meta = df[df['Model']=='human'].groupby(['SceneFullName'])
columns = ['sub-01','sub-02', 'sub-03', 'sub-05', 'sub-06', 'delta_mean' ]
indexs = list(df_meta.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

order_4phases = ["Early discovery", "Late discovery", "Early practice", "Late practice"]
phase_to_human = {p: i % 4 for i, p in enumerate(order_4phases)}

for scene, df_scene in df_meta:

    df_clearance = df_scene.groupby(["Subject", "Learning_Phase"])["Average_speed"].mean().reset_index(name="Average_speed")
    df_delta_tot = utils.compute_delta_tot(df_clearance, phase_to_human, 'Average_speed')

    delta_mean = {'delta_mean': df_delta_tot['delta_tot'].mean().item()}
    delta_sub = dict(zip(df_delta_tot['Subject'], df_delta_tot['delta_tot']))
    row = {**delta_sub, **delta_mean}

    scene_name = scene if isinstance(scene, str) else scene[0]

    df_deltas_tot.loc[scene_name] = row

df_deltas_tot = df_deltas_tot.sort_values('delta_mean', ascending=True)
hum_bot_spd = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(range)

Returns hum_bot_spd: a list of the bottom $range scenes in terms of delta average speed for the humans.

## Bottom Mean MAD for Humans

In [ ]:
df_var = df_variables[df_variables['Subject'].str.startswith('sub-')].groupby(['SceneFullName'])
columns = ['sub-01','sub-02', 'sub-03', 'sub-05', 'sub-06', 'MAD_mean' ]
indexs = list(df_var.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

for scene, df_scene in df_var:

    df_delta_tot = utils.get_mads(df_scene).groupby('Subject')["MAD_mean"].mean().reset_index(name='MAD_mean')
    delta_mean = {'MAD_mean': df_delta_tot['MAD_mean'].mean().item()}
    delta_sub = dict(zip(df_delta_tot['Subject'], df_delta_tot['MAD_mean']))
    row = {**delta_sub, **delta_mean}

    scene_name = scene if isinstance(scene, str) else scene[0]

    df_deltas_tot.loc[scene_name] = row

df_deltas_tot = df_deltas_tot.sort_values('MAD_mean', ascending=True)
hum_bot_mad = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(range)

Returns hum_bot_mad: a list of the bottom $range scenes in terms of mean MAD for the humans.

## Top Clearance Deltas for PPO (delta total)

In [66]:
df_meta = df[df['Model'].str.startswith('ep')].groupby(['SceneFullName'])
columns = ['delta_mean']
indexs = list(df_meta.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

order_4phases = ['ep-20', 'ep-2000', 'ep-4000', 'ep-6000', 'ep-8000']
phase_to_human = {p: i % 5 for i, p in enumerate(order_4phases)}

for scene, df_scene in df_meta:
    df_clearance = df_scene.groupby(["Subject", "Learning_Phase"])["Cleared"].mean().reset_index(name="Cleared")
    df_delta_tot = utils.compute_delta_tot(df_clearance, phase_to_human, 'Cleared')
    delta_sub = {'delta_mean': df_delta_tot['delta_tot'].values[0]}
    scene_name = scene if isinstance(scene, str) else scene[0]
    df_deltas_tot.loc[scene_name] = delta_sub

df_deltas_tot = df_deltas_tot.sort_values('delta_mean', ascending=False)
ppo_top_clr = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(5)

Returns ppo_top_clr: a list of the top $range scenes in terms of delta clearance for the ppo.

## Top Speed Deltas for PPO (delta total)

In [ ]:
df_meta = df[df['Model'].str.startswith('ep')].groupby(['SceneFullName'])
columns = ['delta_mean']
indexs = list(df_meta.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

order_4phases = ['ep-20', 'ep-2000', 'ep-4000', 'ep-6000', 'ep-8000']
phase_to_human = {p: i % 5 for i, p in enumerate(order_4phases)}

for scene, df_scene in df_meta:
    df_clearance = df_scene.groupby(["Subject", "Learning_Phase"])["Average_speed"].mean().reset_index(name="Average_speed")
    df_delta_tot = utils.compute_delta_tot(df_clearance, phase_to_human, 'Average_speed')
    delta_sub = {'delta_mean': df_delta_tot['delta_tot'].values[0]}
    scene_name = scene if isinstance(scene, str) else scene[0]
    df_deltas_tot.loc[scene_name] = delta_sub

df_deltas_tot = df_deltas_tot.sort_values('delta_mean', ascending=False)
ppo_top_spd = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(5)

Returns ppo_top_spd: a list of the top $range scenes in terms of delta average speed for the ppo.

## Top Mean MAD for the PPO

In [ ]:
df_var = df_variables[df_variables['Subject'].str.startswith('ppo')].groupby(['SceneFullName'])
columns = [ 'MAD_mean' ]
indexs = list(df_var.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

for scene, df_scene in df_var:

    df_delta_tot = utils.get_mads(df_scene).groupby('Subject')["MAD_mean"].mean().reset_index(name='MAD_mean')
    delta_mean = {'MAD_mean': df_delta_tot['MAD_mean'].values[0]}
    scene_name = scene if isinstance(scene, str) else scene[0]
    df_deltas_tot.loc[scene_name] = delta_mean

df_deltas_tot = df_deltas_tot.sort_values('MAD_mean', ascending=False)
ppo_top_mad = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(5)

Returns ppo_top_mad: a list of the top $range scenes in terms of mean MAD for the ppo.

## Bottom Clearance Deltas for PPO (delta total)

In [75]:
df_meta = df[df['Model'].str.startswith('ep')].groupby(['SceneFullName'])
columns = ['delta_mean']
indexs = list(df_meta.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

order_4phases = ['ep-20', 'ep-2000', 'ep-4000', 'ep-6000', 'ep-8000']
phase_to_human = {p: i % 5 for i, p in enumerate(order_4phases)}

for scene, df_scene in df_meta:
    df_clearance = df_scene.groupby(["Subject", "Learning_Phase"])["Cleared"].mean().reset_index(name="Cleared")
    df_delta_tot = utils.compute_delta_tot(df_clearance, phase_to_human, 'Cleared')
    delta_sub = {'delta_mean': df_delta_tot['delta_tot'].values[0]}
    scene_name = scene if isinstance(scene, str) else scene[0]
    df_deltas_tot.loc[scene_name] = delta_sub

df_deltas_tot = df_deltas_tot.sort_values('delta_mean', ascending=True)
ppo_bot_clr = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(5)

Returns ppo_bot_clr: a list of the bottom $range scenes in terms of delta clearance for the ppo.

## Bottom Speed Deltas for PPO (delta total)

In [ ]:
df_meta = df[df['Model'].str.startswith('ep')].groupby(['SceneFullName'])
columns = ['delta_mean']
indexs = list(df_meta.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

order_4phases = ['ep-20', 'ep-2000', 'ep-4000', 'ep-6000', 'ep-8000']
phase_to_human = {p: i % 5 for i, p in enumerate(order_4phases)}

for scene, df_scene in df_meta:
    df_clearance = df_scene.groupby(["Subject", "Learning_Phase"])["Average_speed"].mean().reset_index(name="Average_speed")
    df_delta_tot = utils.compute_delta_tot(df_clearance, phase_to_human, 'Average_speed')
    delta_sub = {'delta_mean': df_delta_tot['delta_tot'].values[0]}
    scene_name = scene if isinstance(scene, str) else scene[0]
    df_deltas_tot.loc[scene_name] = delta_sub

df_deltas_tot = df_deltas_tot.sort_values('delta_mean', ascending=True)
ppo_bot_spd = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(5)

Returns ppo_bot_spd: a list of the bottom $range scenes in terms of delta average speed for the ppo.

## Bottom Mean MAD for the PPO

In [76]:
df_var = df_variables[df_variables['Subject'].str.startswith('ppo')].groupby(['SceneFullName'])
columns = [ 'MAD_mean' ]
indexs = list(df_var.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)

for scene, df_scene in df_var:

    df_delta_tot = utils.get_mads(df_scene).groupby('Subject')["MAD_mean"].mean().reset_index(name='MAD_mean')
    delta_mean = {'MAD_mean': df_delta_tot['MAD_mean'].values[0]}
    scene_name = scene if isinstance(scene, str) else scene[0]
    df_deltas_tot.loc[scene_name] = delta_mean

df_deltas_tot = df_deltas_tot.sort_values('MAD_mean', ascending=True)
ppo_bot_mad = df_deltas_tot.index[:range].tolist()
#df_deltas_tot.head(5)

Returns ppo_bot_mad: a list of the bottom $range scenes in terms of mean MAD for the ppo.

## Top Clearance Deltas for Imitation Agents (delta total)

In [11]:
df_meta = df[df['Subject'].str.startswith('im')].groupby(['SceneFullName'])
df_imee = df[df['Subject'].str.startswith('im')]
columns = ['im_sub-01','im_sub-02', 'im_sub-03', 'im_sub-05', 'im_sub-06', 'im_delta_mean' ]
indexs = list(df_meta.groups.keys())
df_deltas_tot = pd.DataFrame(columns=columns, index=indexs)
df_imee = df[df['Subject'].str.startswith('im')]

ppos_phases = [['sub-01_epoch=0-step=500', 'sub-01_epoch=0-step=2000', 'sub-01_epoch=0-step=3500', 'sub-01_epoch=0-step=5000', 'sub-01_epoch=0-step=6500'],
                ['sub-02_epoch=0-step=500', 'sub-02_epoch=0-step=3000', 'sub-02_epoch=0-step=5500', 'sub-02_epoch=0-step=8000', 'sub-02_epoch=0-step=10000'],
                ['sub-03_epoch=0-step=500', 'sub-03_epoch=0-step=4000', 'sub-03_epoch=0-step=7500', 'sub-03_epoch=1-step=11408', 'sub-03_epoch=1-step=14908'],
                ['sub-05_epoch=0-step=500', 'sub-05_epoch=0-step=1500', 'sub-05_epoch=0-step=3000', 'sub-05_epoch=0-step=4000', 'sub-05_epoch=0-step=5000'],
                ['sub-06_epoch=0-step=500', 'sub-06_epoch=0-step=2000', 'sub-06_epoch=0-step=4000', 'sub-06_epoch=0-step=5500', 'sub-06_epoch=0-step=7000']]

phase_to_checkpoint = {p: i % 5 for i, p in enumerate([phase for subset in ppos_phases for phase in subset])}


for scene, df_scene in df_meta:
    df_clearance = df_scene.groupby(["Subject", "Learning_Phase"])["Cleared"].mean().reset_index(name="Cleared")
    df_delta_tot = utils.compute_delta_tot(df_clearance, phase_to_checkpoint, 'Cleared')
    delta_mean = {'im_delta_mean': df_delta_tot['delta_tot'].mean().item()}
    delta_sub = dict(zip(df_delta_tot['Subject'], df_delta_tot['delta_tot']))
    row = {**delta_sub, **delta_mean}
    scene_name = scene if isinstance(scene, str) else scene[0]
    df_deltas_tot.loc[scene_name] = row

df_deltas_tot = df_deltas_tot.sort_values('im_delta_mean', ascending=False)
hum_top_clr = df_deltas_tot.index[:range].tolist()
df_deltas_tot.head(5)

,im_sub-01,im_sub-02,im_sub-03,im_sub-05,im_sub-06,im_delta_mean
8-2-14,NaN,NaN,1.0,0.5,0.851852,0.783951
7-3-4,0.892857,0.509434,0.430769,0.657895,0.877193,0.67363
4-2-8,NaN,NaN,NaN,0.666667,NaN,0.666667
2-3-4,0.78,0.363636,0.525,0.644068,0.795455,0.621632
1-1-4,0.804878,0.5625,0.653846,0.833333,0.035714,0.578054


Returns hum_top_clr: a list of the top $range scenes in terms of delta clearance for the humans.

In [36]:
hum_top_inter =  set(hum_top_clr) &  set(hum_top_mad) & set(hum_top_spd) 
hum_top_inter

{'3-3-2', '4-3-2', '4-3-4'}